In [1]:
import numpy as np
import qutip
import pickle
import nevergrad as ng
from CoupledQuantumSystems.drive import DriveTerm, square_pulse_with_rise_fall
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process
from concurrent.futures import ProcessPoolExecutor
# -------------------------------
# Define your parameters and objects
# -------------------------------
EJ = 3
EJoverEC = 6
EJoverEL = 25
EC = EJ / EJoverEC
EL = EJ / EJoverEL

qbt = gfIFQ(EJ=EJ, EC=EC, EL=EL, flux=0, truncated_dim=20)
e_ops = [
    qutip.basis(qbt.truncated_dim, i) * qutip.basis(qbt.truncated_dim, i).dag()
    for i in range(10)
]

element = np.abs(
    qbt.fluxonium.matrixelement_table("n_operator", evals_count=3)[1, 2]
)
freq = (qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]) * 2 * np.pi


def objective(t_tot, amp, w_d,ramp):
    tlist = np.linspace(0, t_tot * (1+ramp), 100)

    initial_states = [
        qutip.basis(qbt.truncated_dim, 1),
        qutip.basis(qbt.truncated_dim, 2),
    ]

    drive_terms = [
        DriveTerm(
            driven_op=qutip.Qobj(qbt.fluxonium.n_operator(energy_esys=True)),
            pulse_shape_func=square_pulse_with_rise_fall,
            pulse_id="pi",
            pulse_shape_args={
                "w_d": w_d,    # No extra 2pi factor
                "amp": amp,    # No extra 2pi factor
                "t_square": t_tot * (1-ramp),
                "t_rise": t_tot * ramp,
            },
        )
    ]

    # Solve the dynamics for each initial state
    results = []
    for init_state in initial_states:
        res = ODEsolve_and_post_process(
            y0=init_state,
            tlist=tlist,
            static_hamiltonian=qbt.diag_hamiltonian,
            drive_terms=drive_terms,
            e_ops=e_ops,
            print_progress=False,
        )
        results.append(res)

    # Calculate final populations
    # (We want to transfer |1> -> |2> and |2> -> |1>.)
    one_minus_pop2 = abs(1 - results[0].expect[2][-1])  # from state |1> to |2>
    one_minus_pop1 = abs(1 - results[1].expect[1][-1])  # from state |2> to |1>

    return one_minus_pop2 + one_minus_pop1

# ---------------------------------------------------------------------
# SET UP NEVERGRAD INSTRUMENTATION
# ---------------------------------------------------------------------
# We'll specify a search space for each of t_tot, amp, w_d.
# For example, let's guess t_tot is in [1, 300].
# We'll assume amp in [0, 0.1], w_d in [freq*0.9, freq*1.1] as an example.
# Adjust these bounds to your scenario.


# t_tot_list = np.linspace(50, 250, 21)
# Start from 160
t_tot_list = [220,230,240,250]
for t_tot in t_tot_list:
    # init_amp = np.pi / element / t_tot / 3.1415 / 2
    init_wd = qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]
    amp_guess = 50/t_tot * 22
    parametrization = ng.p.Instrumentation(
        amp=ng.p.Scalar(init=amp_guess, lower=amp_guess/5, upper=amp_guess*5),        # or choose your own guess
        w_d=ng.p.Log(init=init_wd, lower=0.003, upper=0.02),
        ramp=ng.p.Scalar(init=0.15, lower=1e-10, upper=0.5),
    )
    # ---------------------------------------------------------------------
    # CHOOSE THE OPTIMIZER
    # ---------------------------------------------------------------------
    # Some popular choices: "CMA", "TwoPointsDE", "OnePlusOne", "PSO", "NGOpt", etc.
    # We'll pick CMA as an example. We'll also set a budget (# of function evaluations)
    # and a concurrency level (num_workers) for parallelizing the objective calls.
    budget = 1000
    num_workers = 20
    optimizer = ng.optimizers.CMA(parametrization=parametrization,
                                budget=budget,
                                num_workers=num_workers)
    # ---------------------------------------------------------------------
    # RUN THE OPTIMIZATION
    # (Nevergrad automatically parallelizes if num_workers > 1.)
    # ---------------------------------------------------------------------
    def nevergrad_objective( amp, w_d,ramp):
        return objective(t_tot=t_tot, amp=amp, w_d=w_d,ramp=ramp)
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        recommendation = optimizer.minimize(nevergrad_objective,batch_mode=True,verbosity=2,executor=executor)

    # Extract best values
    best_amp = recommendation.kwargs["amp"]
    best_w_d = recommendation.kwargs["w_d"]
    best_ramp = recommendation.kwargs["ramp"]

    print(f"Best amp: {best_amp}, best w_d: {best_w_d}, best ramp: {best_ramp}")

    # Optionally, store results in a pickle
    results_dict = {
        "best_amp": best_amp,
        "best_w_d": best_w_d,
        "best_ramp": best_ramp,
        "recommendation": recommendation,  # the full nevergrad result object
    }

    with open(f"nevergrad_optimized_results_{t_tot}.pkl", "wb") as f:
        pickle.dump(results_dict, f)


Launching 20 jobs with new suggestions
Updating fitness with value 1.9416518918439287
Updating fitness with value 1.4786642195021698
980 remaining budget and 18 running jobs
Current pessimistic best is: MultiValue<mean: 1.4786642195021698, count: 1, parameter: Instrumentation(Tuple(),Dict(amp=Scalar{Cl(1,25,b)}[sigma=Scalar{exp=2.03}],ramp=Scalar{Cl(1e-10,0.5,b)}[sigma=Scalar{exp=2.03}],w_d=Log{Cl(-18.372511079725303,-12.372511079725303,b),exp=1.37})):((), {'amp': 1.6347958842747707, 'w_d': 0.0034613333453793953, 'ramp': 0.11163668947489254})>


 /home/kai/miniconda3/envs/test/lib/python3.10/site-packages/nevergrad/common/tools.py: 107

Updating fitness with value 1.7261812259007154
Updating fitness with value 1.5531362919626055
Updating fitness with value 1.7781865590522767
Updating fitness with value 1.8337670874664371
Updating fitness with value 1.9436282435436651
Updating fitness with value 1.5651068911538362
Updating fitness with value 1.7237091158394284
Updating fitness with value 1.7650792711961043
Updating fitness with value 1.8597700552584298
Updating fitness with value 1.7726967040539265
Updating fitness with value 1.5407066317198361
Updating fitness with value 1.518763776681115
Updating fitness with value 1.783739205516626
Updating fitness with value 1.794131843982214
Updating fitness with value 1.843794779819537
Updating fitness with value 1.5697116951558054
Updating fitness with value 1.7232824185294162
Updating fitness with value 1.9484332996508793
980 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 1.4786642195021698, count: 1, parameter: Instrumentation(Tuple(),Dict(am

 /home/kai/miniconda3/envs/test/lib/python3.10/site-packages/nevergrad/common/tools.py: 107

Updating fitness with value 1.835159883308239
Updating fitness with value 1.751368314263392
Updating fitness with value 1.800325042524289
Updating fitness with value 1.538751025603529
Updating fitness with value 1.8483223337258006
Updating fitness with value 1.7960582382454091
Updating fitness with value 1.9077394550257631
Updating fitness with value 1.9479490759377391
Updating fitness with value 1.583070346352467
Updating fitness with value 1.450301287866549
Updating fitness with value 1.7852902761094658
Updating fitness with value 1.3780213687170249
Updating fitness with value 1.7739920142183059
Updating fitness with value 1.863386790819618
Updating fitness with value 1.58503652034915
Updating fitness with value 1.839623732750768
Updating fitness with value 1.9843680577310785
980 remaining budget and 0 running jobs
Current pessimistic best is: MultiValue<mean: 1.3780213687170249, count: 1, parameter: Instrumentation(Tuple(),Dict(amp=Scalar{Cl(0.9565217391304348,23.913043478260867,b)}

 /home/kai/miniconda3/envs/test/lib/python3.10/site-packages/nevergrad/common/tools.py: 107

Updating fitness with value 1.766408391783052
Updating fitness with value 1.934905904576426
Updating fitness with value 1.9280556551371837
Updating fitness with value 1.4097132476309777
Updating fitness with value 1.8311862498513343
Updating fitness with value 1.9112488314944738
Updating fitness with value 1.9123705936110913
Updating fitness with value 1.6958109715017884
Updating fitness with value 1.7233544712005067
Updating fitness with value 1.7521501069292642
Updating fitness with value 1.6789941898968164
Updating fitness with value 1.6230249355355708
Updating fitness with value 1.4306443021683515
Updating fitness with value 1.4063394688287967
Updating fitness with value 1.7469607854351166
Updating fitness with value 1.937687997845651
Updating fitness with value 1.1778877254245779
980 remaining budget and 2 running jobs
Current pessimistic best is: MultiValue<mean: 1.1778877254245779, count: 1, parameter: Instrumentation(Tuple(),Dict(amp=Scalar{Cl(0.9166666666666667,22.916666666666

 /home/kai/miniconda3/envs/test/lib/python3.10/site-packages/nevergrad/common/tools.py: 107

Updating fitness with value 1.8362442294236556
Updating fitness with value 1.9985368045919567
Updating fitness with value 1.8642779017771254
Updating fitness with value 1.9941132948218874
Updating fitness with value 1.7539617906900082
Updating fitness with value 1.6275759657485107
Updating fitness with value 1.287570423522054
Updating fitness with value 1.853509837754091
Updating fitness with value 1.8260755536555053
980 remaining budget and 10 running jobs
Current pessimistic best is: MultiValue<mean: 1.287570423522054, count: 1, parameter: Instrumentation(Tuple(),Dict(amp=Scalar{Cl(0.8800000000000001,22,b)}[sigma=Scalar{exp=2.03}],ramp=Scalar{Cl(1e-10,0.5,b)}[sigma=Scalar{exp=2.03}],w_d=Log{Cl(-18.372511079725303,-12.372511079725303,b),exp=1.37})):((), {'amp': 2.9522939588758788, 'w_d': 0.003300410188344226, 'ramp': 0.19332908186096523})>
Updating fitness with value 1.967397406414669
980 remaining budget and 9 running jobs
Current pessimistic best is: MultiValue<mean: 1.2875704235220